In [2]:
from IPython.display import display
import pandas as pd
import seaborn as sns
import numpy as np
pd.options.display.max_columns = None

In [1]:
REDUCE_DEFAULT = "OTHER"

class FactorReducer():
    def __init__(self):
        self.factors_to_keep = []
        
    def fit(self, series, max_factors, reduce_to=REDUCE_DEFAULT):
        self.factors_to_keep = series.value_counts().keys()[0:max_factors]
        self.reduce_to = REDUCE_DEFAULT
        
    def transform(self, series):
        new_column = series.copy(deep=True)
        indices_for_other = np.logical_not(series.isin(self.factors_to_keep))
        new_column.loc[indices_for_other] = self.reduce_to
        return new_column

In [3]:
train_features = pd.read_csv("train_features.csv")
train_target = pd.read_csv("train_target.csv").fillna("Empty")
test_features = pd.read_csv("test_features.csv").fillna("Empty")

#### Join into one training dataset
loaded_train = train_features.set_index("id").join(train_target.set_index("id"))

In [4]:
TARGET = "status_group"

NUMERIC_FEATURES = [
    'amount_tsh',
    'gps_height',
    'longitude',
    'latitude',
    'population'
]

CATEGORICAL_FEATURES = [
    'construction_year',
    'funder',
    'installer',
    'basin',
    'region',
    'public_meeting',
    'scheme_management',
    'permit',
    'extraction_type',
    'extraction_type_group',
    'extraction_type_class',
    'management',
    'management_group',
    'payment',
    'payment_type',
    'water_quality',
    'quality_group',
    'quantity',
    'quantity_group',
    'source',
    'source_type',
    'source_class',
    'waterpoint_type',
    'waterpoint_type_group'
]

DROP_COLUMNS = [
    "wpt_name",
    "num_private",
    "subvillage",
    "region_code",
    "district_code",
    "lga",
    "ward",
    "recorded_by",
    "scheme_name",
]

CATEGORY_CAPS = {
    "funder": 55,
    "installer": 45,
    "construction_year": 9
}

In [5]:
train = loaded_train.copy(deep=True)
train.drop(DROP_COLUMNS, 1)
test_features.drop(DROP_COLUMNS, 1)

categorical_reducers = {}
for feature, max_factors in CATEGORY_CAPS.items():
    reducer = FactorReducer()
    reducer.fit(train[feature], max_factors)
    categorical_reducers[feature] = reducer

def build_cleaned_frame(input_frame, numerics, categoricals, categorical_reducers):
    category_frame = input_frame[categoricals]
    for feature, reducer in categorical_reducers.items():
        category_frame[feature] = reducer.transform(category_frame[feature])
    cleaned_frame = input_frame[numerics]
    dummy_category_frame = pd.get_dummies(category_frame, prefix=categoricals, prefix_sep="=")
    return cleaned_frame.join(dummy_category_frame)

cleaned_train = build_cleaned_frame(train, NUMERIC_FEATURES, CATEGORICAL_FEATURES, categorical_reducers)
cleaned_test = build_cleaned_frame(test_features, NUMERIC_FEATURES, CATEGORICAL_FEATURES, categorical_reducers)

/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
import datetime
def get_date_frame(df):
    ret_frame = pd.DataFrame(index=df.index)

    dates = pd.DatetimeIndex(df['date_recorded'])
    ret_frame["year_recorded"] = dates.year
    ret_frame["month_recorded"] = dates.month

    dates = df['date_recorded'].astype('datetime64[ns]')
    ret_frame["recorded_days_ago"] = (datetime.date.today() - dates).apply(lambda x: x.days)
    return ret_frame

train_date_frame = get_date_frame(train)
test_date_frame = get_date_frame(test_features)

In [7]:
cleaned_train = cleaned_train.join(train_date_frame)
cleaned_train.to_csv('cleaned_train.csv', index=False)
cleaned_test = cleaned_test.join(test_date_frame)
cleaned_test.to_csv('cleaned_test.csv', index=False)
train[TARGET].to_csv('YS.csv', index=False)

In [8]:
import numpy as np
cols = np.intersect1d(cleaned_train.columns, cleaned_test.columns)

In [9]:
# cleaned_test.loc[cleaned_test['construction_year=2000'].isnull()]
cleaned_test.isnull().sum()

amount_tsh                                     0
gps_height                                     0
longitude                                      0
latitude                                       0
population                                     0
construction_year=0                            0
construction_year=2000                         0
construction_year=2003                         0
construction_year=2006                         0
construction_year=2007                         0
construction_year=2008                         0
construction_year=2009                         0
construction_year=2010                         0
construction_year=2011                         0
construction_year=OTHER                        0
funder=0                                       0
funder=Adb                                     0
funder=Adra                                    0
funder=Amref                                   0
funder=Ces(gmbh)                               0
funder=Danida       

### Initialize xgboost and run it

In [46]:
%%time

from xgboost import XGBClassifier
from sklearn.cross_validation import cross_val_score

SAMPLE_SIZE = 10000
sample = cleaned_train.sample(SAMPLE_SIZE)
sample_target = train[TARGET].loc[sample.index]

#  colsample_bytree |     gamma |   max_depth |   scale_pos_weight
# 0.3124 |    1.7012 |     18.4428 |            20.0331
xgb = XGBClassifier(max_depth=18, colsample_bytree=0.31, gamma=18)
# xgb = XGBClassifier(max_depth=20, colsample_bytree=0.2)
scores = cross_val_score(xgb, cleaned_train, train[TARGET])
# scores = cross_val_score(xgb, sample, sample_target)
print(scores.mean())

0.765824915825
CPU times: user 5min 51s, sys: 1.95 s, total: 5min 53s
Wall time: 5min 54s


In [38]:
sample = cleaned_train.sample(10000)
sample_target = train[TARGET].loc[sample.index]

(10000,)

### Compare time to useing dmatrix

In [31]:
%%time
from sklearn.preprocessing import LabelEncoder
from xgboost import DMatrix, cv
params = {
    'max_depth':20, 
    'colsample_bytree':0.2,
    'objective': 'multi:softmax',
    'num_class': 3
}

le = LabelEncoder()
le.fit(train[TARGET])

dmatrix = DMatrix(cleaned_train, le.transform(train[TARGET]))
xgb = XGBClassifier(max_depth=20, colsample_bytree=0.2)
cv_xgb = cv(params, dtrain = dmatrix, num_boost_round = 3000, nfold = 3,
                metrics = ['merror'], # Make sure you enter metrics inside a list or you may encounter issues!
                early_stopping_rounds = 100) # Look for early stopping that minimizes error

KeyboardInterrupt: 

### Set up Bayesian Optimization

In [48]:
from bayes_opt import bayesian_optimization

def xgboost_eval(gamma, max_depth, colsample_bytree):
    fit_params={
        'eval_metric': 'auc'
    }
    
    max_depth = int(max_depth)
    xgbc = XGBClassifier(
        gamma=gamma,
        max_depth=max_depth, 
        colsample_bytree=colsample_bytree 
#         scale_pos_weight=scale_pos_weight
    )
#     SAMPLE_SIZE = 10000
#     sample = cleaned_train.sample(SAMPLE_SIZE)
#     sample_target = train[TARGET].loc[sample.index]
    scores = cross_val_score(xgbc, cleaned_train, train[TARGET], n_jobs=-1)
#     scores = cross_val_score(xgbc, sample, sample_target, n_jobs=-1)
    return scores.mean()

xgboostBO = bayesian_optimization.BayesianOptimization(xgboost_eval, {
        'max_depth': (3, 25),
        'colsample_bytree': (.3, .9),
        'gamma':(0,5)
    })

xgboostBO.maximize(init_points=5, n_iter=100)

#  colsample_bytree |     gamma |   max_depth |   scale_pos_weight
# 0.3124 |    1.7012 |     18.4428 |            20.0331
# 0.8429 |    0.3241 |      9.0384 |            24.9536
# 0.3000 |    0.5428 |     16.2778 |            14.6219
# 0.4983 |    1.3606 |      4.1616 |            38.6310

Initialization
----------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_depth | 
    1 | 05m07s |    0.80480 |             0.7357 |    3.5135 |     23.9064 | 
    2 | 02m17s |    0.79921 |             0.7586 |    2.9604 |     11.5544 | 
    3 | 02m49s |    0.80239 |             0.8697 |    2.2864 |     12.7882 | 
    4 | 02m30s |    0.80715 |             0.3881 |    2.7874 |     20.0750 | 
    5 | 02m00s |    0.80652 |             0.3648 |    2.5791 |     17.9346 | 


/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'nit': 5, 'warnflag': 2, 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 49, 'grad': array([ -2.78404041e-05])}
  " state: %s" % convergence_dict)


Bayesian Optimization
----------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_depth | 
    6 | 02m29s |    0.80877 |             0.3000 |    0.0000 |     23.4605 | 


/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'nit': 5, 'warnflag': 2, 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 53, 'grad': array([ -5.50741461e-05])}
  " state: %s" % convergence_dict)


    7 | 04m18s |    0.80791 |             0.8339 |    0.6502 |     18.1128 | 
    8 | 05m07s |    0.80862 |             0.8940 |    1.3836 |     20.9226 | 


...

  102 | 02m08s |    0.80948 |             0.3260 |    1.6645 |     19.4979 | 


/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'nit': 4, 'warnflag': 2, 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 47, 'grad': array([ 0.00019447])}
  " state: %s" % convergence_dict)


  103 | 02m25s |    0.81145 |             0.3006 |    0.8881 |     23.1042 | 


/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'nit': 6, 'warnflag': 2, 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'grad': array([-0.00636503])}
  " state: %s" % convergence_dict)
/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:308: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


  104 | 06m13s |    0.80364 |             0.8701 |    0.0255 |     24.7043 | 
  105 | 04m00s |    0.79884 |             0.8336 |    4.7494 |     17.0806 | 


In [51]:
xgboostBO.res

{'all': {'params': [{'colsample_bytree': 0.30000002220448635,
    'gamma': 3.6341927761870185e-08,
    'max_depth': 23.460506818277675},
   {'colsample_bytree': 0.83385209865984888,
    'gamma': 0.65015169270178041,
    'max_depth': 18.112827976298},
   {'colsample_bytree': 0.89395937004850978,
    'gamma': 1.3836235580555909,
    'max_depth': 20.922624295670243},
   {'colsample_bytree': 0.3920300904631901,
    'gamma': 0.030808011925219189,
    'max_depth': 22.455419684869572},
   {'colsample_bytree': 0.73030740467385913,
    'gamma': 0.21732635500365194,
    'max_depth': 19.615724255488086},
   {'colsample_bytree': 0.60576367715125368,
    'gamma': 0.82402997416723145,
    'max_depth': 3.4391020511722443},
   {'colsample_bytree': 0.814019517179429,
    'gamma': 0.72508668949647581,
    'max_depth': 22.407703306414675},
   {'colsample_bytree': 0.37833744486630255,
    'gamma': 0.13066941299307086,
    'max_depth': 16.770737230444322},
   {'colsample_bytree': 0.56009025234669352,
    '

In [14]:
%%time
from xgboost import XGBClassifier

xgb_out = XGBClassifier(colsample_bytree=0.309, gamma=0.795, max_depth=18)
xgb_out.fit(cleaned_train[cols], train[TARGET])

CPU times: user 3min 31s, sys: 1.84 s, total: 3min 33s
Wall time: 3min 36s


In [225]:
xgb_cross_val = XGBClassifier(colsample_bytree=0.309, gamma=0.795, max_depth=24)
# scores = cross_val_score(xgb_cross_val, cleaned_train[cols], train[TARGET], cv=5, n_jobs=-1)
scores = cross_val_score(xgb_cross_val, centered, train[TARGET], cv=5, n_jobs=-1)

print(scores.mean())

"""0.815336592481
"""

0.816195120244


'0.815336592481\n'

In [15]:
predictions = xgb_out.predict(cleaned_test[cols])
out_frame = pd.DataFrame({
        "id": test_features["id"],
        "status_group": predictions
    })
out_frame.to_csv("bayes_opt_xgb_out.csv", index=False)

In [13]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from xgboost import XGBClassifier

rfc = RandomForestClassifier(n_estimators=100, max_depth=22)
xgb = XGBClassifier(colsample_bytree=0.309, gamma=0.795, max_depth=18)

voter = VotingClassifier(estimators=[
        ('xgb', xgb),
        ('rfc', rfc),
    ], voting="soft")
scores = cross_val_score(voter, cleaned_train[cols], train[TARGET], cv=5, n_jobs=-1)
print(scores.mean())
voter.fit(cleaned_train[cols], train[TARGET])
voter.predict(cleaned_test[cols])
out_frame = pd.DataFrame({
        "id": test_features["id"],
        "status_group": predictions
    })
out_frame.to_csv("voter_bayes_opt_xgb_rfc2.csv", index=False)

"""
0.814865190752
"""

/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-

0.814865190752


NameError: name 'predictions' is not defined

In [17]:
predictions = voter.fit(cleaned_train[cols], train[TARGET])
out_frame = pd.DataFrame({
        "id": test_features["id"],
        "status_group": predictions
    })
out_frame.to_csv("voter_bayes_opt_xgb_rfc.csv", index=False)

### Let's make a voter

In [223]:
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Make a bunch of xgboosts
COL = 'colsample_bytree'
GAM = "gamma"
MD  = 'max_depth'
xgb_settings = [
    {
        COL:0.309,
        GAM: 0.795,
        MD: 18
    },
    {
        COL:0.8892,
        GAM: 0.3534,
        MD: 12
    },
    {
        COL:0.3356,
        GAM: 2.6913,
        MD: 9
    },
    {
        COL:0.8892,
        GAM: 0.3534,
        MD: 12
    },
    {
        COL:0.3575,
        GAM: 0.9201,
        MD: 19
    }
]



# rfc_most_deep = RandomForestClassifier(n_estimators=100, max_depth=30)
# rfc_even_more_deep = RandomForestClassifier(n_estimators=100, max_depth=26)
# rfc_more_deep = RandomForestClassifier(n_estimators=100, max_depth=22)
# rfc_deep = RandomForestClassifier(n_estimators=100, max_depth=20)
# rfc_shallow = RandomForestClassifier(n_estimators=100, max_depth=18)

# rfc_estimators = [
#         ('shallow_rfc', rfc_shallow),
#         ('deep_rfc', rfc_deep),
#         ('more_deep_rfc', rfc_more_deep),
#         ('more_even_more_rfc', rfc_even_more_deep),
#         ('more_most_rfc', rfc_most_deep),
#     ]

xgbs = {}
for ii, xgb_setting in enumerate(xgb_settings):
    xgbs[ii] = XGBClassifier(colsample_bytree=xgb_setting[COL], gamma=xgb_setting[GAM], max_depth=xgb_setting[MD])
    
# put them together with a voting classifier
xgb_voter =  VotingClassifier(estimators=list(xgbs.items()), voting='soft')
mixed_estimators = list(xgbs.items()) + rfc_estimators
mixed_voter = VotingClassifier(estimators=mixed_estimators, voting="soft")

xgb_scores = cross_val_score(xgb_voter, cleaned_train[cols], train[TARGET], n_jobs=-1)
mixed_scores = cross_val_score(mixed_voter, cleaned_train[cols], train[TARGET], n_jobs=-1)

print("xgb voter: %s" % xgb_scores.mean())
print("mixed voter: %s" % mixed_scores.mean())

"""
xgb voter: 0.808097643098
mixed voter: 0.809646464646
"""


/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


NameError: name 'rfc_estimators' is not defined

In [35]:
%%time
mixed_voter.fit(cleaned_train[cols], train[TARGET])
predictions = voter.predict(cleaned_test[cols])
output = pd.DataFrame({
        'id': test_features['id'],
        'status_group': predictions
    })
output.to_csv('mixed_voter_xgb_rfc.csv', index=False)

KeyboardInterrupt: 

In [164]:
cleaned_test.isnull().sum()

amount_tsh                                     0
gps_height                                     0
longitude                                      0
latitude                                       0
population                                     0
construction_year=0                            0
construction_year=2000                         0
construction_year=2003                         0
construction_year=2006                         0
construction_year=2007                         0
construction_year=2008                         0
construction_year=2009                         0
construction_year=2010                         0
construction_year=2011                         0
construction_year=OTHER                        0
funder=0                                       0
funder=Adb                                     0
funder=Adra                                    0
funder=Amref                                   0
funder=Ces(gmbh)                               0
funder=Danida       

In [204]:
rfc_most_deep = RandomForestClassifier(n_estimators=100, max_depth=26)
rfc_even_more_deep = RandomForestClassifier(n_estimators=100, max_depth=24)
rfc_more_deep = RandomForestClassifier(n_estimators=100, max_depth=22)
rfc_deep = RandomForestClassifier(n_estimators=100, max_depth=20)
rfc_shallow = RandomForestClassifier(n_estimators=100, max_depth=18)
voter = VotingClassifier(estimators=[
        ('shallow_rfc', rfc_shallow),
        ('deep_rfc', rfc_deep),
        ('more_deep_rfc', rfc_more_deep),
        ('more_even_more_rfc', rfc_even_more_deep),
        ('more_most_rfc', rfc_most_deep),
    ], voting="soft")


In [36]:
%%time
# 40 0.747087542088 md=10
# 100 0.794511784512 md=20
# 40 0.798097643098 md=20
# 30 0.795757575758 md=30
from sklearn.ensemble import RandomForestClassifier

rfc_most_deep = RandomForestClassifier(n_estimators=100, max_depth=30)
rfc_even_more_deep = RandomForestClassifier(n_estimators=100, max_depth=26)
rfc_more_deep = RandomForestClassifier(n_estimators=100, max_depth=22)
rfc_deep = RandomForestClassifier(n_estimators=100, max_depth=20)
rfc_shallow = RandomForestClassifier(n_estimators=100, max_depth=18)

scores_most_deep = cross_val_score(rfc_most_deep, cleaned_train[cols], train[TARGET], cv=5, n_jobs=-1)
scores_even_more_deep = cross_val_score(rfc_even_more_deep, cleaned_train[cols], train[TARGET], cv=5, n_jobs=-1)
scores_more_deep = cross_val_score(rfc_more_deep, cleaned_train[cols], train[TARGET], cv=5, n_jobs=-1)
scores_deep = cross_val_score(rfc_deep, cleaned_train[cols], train[TARGET], cv=5, n_jobs=-1)
scores_shallow = cross_val_score(rfc_shallow, cleaned_train[cols], train[TARGET], cv=5, n_jobs=-1)

scores_voter = cross_val_score(voter, cleaned_train[cols], train[TARGET], n_jobs=-1, cv=5)
print("shallow: %s" % scores_shallow.mean())
print("deep: %s" % scores_deep.mean())
print("more deep: %s" % scores_more_deep.mean())
print("even more deep: %s" % scores_even_more_deep.mean())
print("most deep: %s" % scores_most_deep.mean())
print("voter: %s" % scores_voter.mean())

TypeError: can't pickle dict_items objects

In [26]:
from bayes_opt import bayesian_optimization

def eval_rfc(n_years, n_funders, n_installers):
    train = loaded_train.copy(deep=True)
    train.drop(DROP_COLUMNS, 1)
    test_features.drop(DROP_COLUMNS, 1)

    category_caps = {
        "funder": int(n_funders),
        "installer": int(n_installers),
        "construction_year": int(n_years)
    }
    categorical_reducers = {}
    for feature, max_factors in category_caps.items():
        reducer = FactorReducer()
        reducer.fit(train[feature], max_factors)
        categorical_reducers[feature] = reducer
        train[feature] = reducer.transform(train[feature])
        
    cleaned_train = build_cleaned_frame(train, NUMERIC_FEATURES, CATEGORICAL_FEATURES, categorical_reducers)
    cleaned_test = build_cleaned_frame(test_features, NUMERIC_FEATURES, CATEGORICAL_FEATURES, categorical_reducers)
    cleaned_train = cleaned_train.join(train_date_frame)
    cleaned_test = cleaned_test.join(test_date_frame)
    
    rfc = RandomForestClassifier(n_estimators=20, max_depth=20)
    scores = cross_val_score(rfc, cleaned_train[cleaned_test.columns], train[TARGET])
    return scores.mean()

bopt = bayesian_optimization.BayesianOptimization(eval_rfc, {
        'n_years': (1, 30),
        'n_funders': (1, 100),
        'n_installers': (1, 100)
    })

bopt.maximize(init_points=5, n_iter=100)

Initialization
------------------------------------------------------------------------
 Step |   Time |      Value |   n_funders |   n_installers |   n_years | 


/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


NameError: name 'RandomForestClassifier' is not defined

In [23]:
from keras.utils import np_utils

le = LabelEncoder()
le.fit(train[TARGET])
encoded_Y = le.transform(train[TARGET])
dummy_y = np_utils.to_categorical(encoded_Y)

In [83]:
train[TARGET].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [70]:
centered = cleaned_train[cols] - cleaned_train[cols].mean()
centered = centered / centered.std()

In [84]:
centered.head(5)

,amount_tsh,basin=Internal,basin=Lake Nyasa,basin=Lake Rukwa,basin=Lake Tanganyika,basin=Lake Victoria,basin=Pangani,basin=Rufiji,basin=Ruvuma / Southern Coast,basin=Wami / Ruvu,construction_year=0,construction_year=2000,construction_year=2003,construction_year=2006,construction_year=2007,construction_year=2008,construction_year=2009,construction_year=2010,construction_year=2011,construction_year=OTHER,extraction_type=afridev,extraction_type=cemo,extraction_type=climax,extraction_type=gravity,extraction_type=india mark ii,extraction_type=india mark iii,extraction_type=ksb,extraction_type=mono,extraction_type=nira/tanira,extraction_type=other,extraction_type=other - play pump,extraction_type=other - rope pump,extraction_type=other - swn 81,extraction_type=submersible,extraction_type=swn 80,extraction_type=walimi,extraction_type=windmill,extraction_type_class=gravity,extraction_type_class=handpump,extraction_type_class=motorpump,extraction_type_class=other,extraction_type_class=rope pump,extraction_type_class=submersible,extraction_type_class=wind-powered,extraction_type_group=afridev,extraction_type_group=gravity,extraction_type_group=india mark ii,extraction_type_group=india mark iii,extraction_type_group=mono,extraction_type_group=nira/tanira,extraction_type_group=other,extraction_type_group=other handpump,extraction_type_group=other motorpump,extraction_type_group=rope pump,extraction_type_group=submersible,extraction_type_group=swn 80,extraction_type_group=wind-powered,funder=0,funder=Adb,funder=Adra,funder=Amref,funder=Ces(gmbh),funder=Danida,funder=Ded,funder=Dh,funder=Dhv,funder=District Council,funder=Dmdd,funder=Dwe,funder=Dwsp,funder=Fini Water,funder=Finw,funder=Fw,funder=Germany Republi,funder=Go,funder=Government Of Tanzania,funder=Hesawa,funder=Hifab,funder=Isf,funder=Jaica,funder=Jica,funder=Kiliwater,funder=Kkkt,funder=Kkkt_makwale,funder=Lawatefuka Water Supply,funder=Lga,funder=Magadini-makiwaru Water,funder=Ministry Of Water,funder=Mission,funder=Netherlands,funder=Norad,funder=OTHER,funder=Oxfam,funder=Oxfarm,funder=Plan Int,funder=Private,funder=Private Individual,funder=Rc,funder=Rc Church,funder=Roman,funder=Rudep,funder=Rural Water Supply And Sanitat,funder=Rwssp,funder=Shipo,funder=Tasaf,funder=Tcrs,funder=Unicef,funder=W.B,funder=Water,funder=Wateraid,funder=World Bank,funder=World Vision,funder=Wsdp,gps_height,installer=0,installer=ACRA,installer=AMREF,installer=CES,installer=Central government,installer=Commu,installer=Community,installer=DANID,installer=DANIDA,installer=DH,installer=DW,installer=DWE,installer=DWSP,installer=Da,installer=Distri,installer=District Council,installer=District council,installer=Dmdd,installer=FinW,installer=Gove,installer=Gover,installer=Government,installer=HESAWA,installer=Hesawa,installer=Idara ya maji,installer=KKKT,installer=Kiliwater,installer=LGA,installer=Lawatefuka water sup,installer=Magadini-Makiwaru wa,installer=NORAD,installer=OTHER,installer=OXFAM,installer=RC,installer=RWE,installer=SEMA,installer=Sengerema Water Department,installer=TASAF,installer=TCRS,installer=TWESA,installer=UNICEF,installer=Villagers,installer=WEDECO,installer=WU,installer=World Vision,installer=World vision,latitude,longitude,management=company,management=other,management=other - school,management=parastatal,management=private operator,management=trust,management=unknown,management=vwc,management=water authority,management=water board,management=wua,management=wug,management_group=commercial,management_group=other,management_group=parastatal,management_group=unknown,management_group=user-group,month_recorded,payment=never pay,payment=other,payment=pay annually,payment=pay monthly,payment=pay per bucket,payment=pay when scheme fails,payment=unknown,payment_type=annually,payment_type=monthly,payment_type=never pay,payment_type=on failure,payment_type=other,payment_type=per bucket,payment_type=unknown,permit=False,permit=True,population,public_meeting=False,public_meeting=True,quality_

In [111]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(centered, train[TARGET], test_size=0.2, random_state=0)

In [120]:
X_train.

,amount_tsh,basin=Internal,basin=Lake Nyasa,basin=Lake Rukwa,basin=Lake Tanganyika,basin=Lake Victoria,basin=Pangani,basin=Rufiji,basin=Ruvuma / Southern Coast,basin=Wami / Ruvu,construction_year=0,construction_year=2000,construction_year=2003,construction_year=2006,construction_year=2007,construction_year=2008,construction_year=2009,construction_year=2010,construction_year=2011,construction_year=OTHER,extraction_type=afridev,extraction_type=cemo,extraction_type=climax,extraction_type=gravity,extraction_type=india mark ii,extraction_type=india mark iii,extraction_type=ksb,extraction_type=mono,extraction_type=nira/tanira,extraction_type=other,extraction_type=other - play pump,extraction_type=other - rope pump,extraction_type=other - swn 81,extraction_type=submersible,extraction_type=swn 80,extraction_type=walimi,extraction_type=windmill,extraction_type_class=gravity,extraction_type_class=handpump,extraction_type_class=motorpump,extraction_type_class=other,extraction_type_class=rope pump,extraction_type_class=submersible,extraction_type_class=wind-powered,extraction_type_group=afridev,extraction_type_group=gravity,extraction_type_group=india mark ii,extraction_type_group=india mark iii,extraction_type_group=mono,extraction_type_group=nira/tanira,extraction_type_group=other,extraction_type_group=other handpump,extraction_type_group=other motorpump,extraction_type_group=rope pump,extraction_type_group=submersible,extraction_type_group=swn 80,extraction_type_group=wind-powered,funder=0,funder=Adb,funder=Adra,funder=Amref,funder=Ces(gmbh),funder=Danida,funder=Ded,funder=Dh,funder=Dhv,funder=District Council,funder=Dmdd,funder=Dwe,funder=Dwsp,funder=Fini Water,funder=Finw,funder=Fw,funder=Germany Republi,funder=Go,funder=Government Of Tanzania,funder=Hesawa,funder=Hifab,funder=Isf,funder=Jaica,funder=Jica,funder=Kiliwater,funder=Kkkt,funder=Kkkt_makwale,funder=Lawatefuka Water Supply,funder=Lga,funder=Magadini-makiwaru Water,funder=Ministry Of Water,funder=Mission,funder=Netherlands,funder=Norad,funder=OTHER,funder=Oxfam,funder=Oxfarm,funder=Plan Int,funder=Private,funder=Private Individual,funder=Rc,funder=Rc Church,funder=Roman,funder=Rudep,funder=Rural Water Supply And Sanitat,funder=Rwssp,funder=Shipo,funder=Tasaf,funder=Tcrs,funder=Unicef,funder=W.B,funder=Water,funder=Wateraid,funder=World Bank,funder=World Vision,funder=Wsdp,gps_height,installer=0,installer=ACRA,installer=AMREF,installer=CES,installer=Central government,installer=Commu,installer=Community,installer=DANID,installer=DANIDA,installer=DH,installer=DW,installer=DWE,installer=DWSP,installer=Da,installer=Distri,installer=District Council,installer=District council,installer=Dmdd,installer=FinW,installer=Gove,installer=Gover,installer=Government,installer=HESAWA,installer=Hesawa,installer=Idara ya maji,installer=KKKT,installer=Kiliwater,installer=LGA,installer=Lawatefuka water sup,installer=Magadini-Makiwaru wa,installer=NORAD,installer=OTHER,installer=OXFAM,installer=RC,installer=RWE,installer=SEMA,installer=Sengerema Water Department,installer=TASAF,installer=TCRS,installer=TWESA,installer=UNICEF,installer=Villagers,installer=WEDECO,installer=WU,installer=World Vision,installer=World vision,latitude,longitude,management=company,management=other,management=other - school,management=parastatal,management=private operator,management=trust,management=unknown,management=vwc,management=water authority,management=water board,management=wua,management=wug,management_group=commercial,management_group=other,management_group=parastatal,management_group=unknown,management_group=user-group,month_recorded,payment=never pay,payment=other,payment=pay annually,payment=pay monthly,payment=pay per bucket,payment=pay when scheme fails,payment=unknown,payment_type=annually,payment_type=monthly,payment_type=never pay,payment_type=on failure,payment_type=other,payment_type=per bucket,payment_type=unknown,permit=False,permit=True,population,public_meeting=False,public_meeting=True,quality_

In [159]:
class_weights = pd.get_dummies(y_train).sum().max() / pd.get_dummies(y_train).sum()
cw = dict(zip(range(3), class_weights.values))
cw

{0: 1.0, 1: 7.482608695652174, 2: 1.4141331142152835}

In [222]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers.convolutional import Convolution1D
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from keras.optimizers import SGD, Adam
from keras.regularizers import WeightRegularizer
from sklearn.metrics import accuracy_score

def keras_model():
#     adam = Adam(lr=0.01)
    model = Sequential() # make a model in a linear stack of layers
    model.add(Dense(200, input_dim=290, activation="relu"))
    model.add(Dropout(.5))
    model.add(Dense(200, activation="relu"))
    model.add(Dropout(.5))
    model.add(Dense(200, activation="relu"))
    model.add(Dropout(.5))
    model.add(Dense(output_dim=3, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer="adagrad", metrics=['accuracy'])
    return model

kc = Sequential()
kc.add(Dense(10, input_shape=(290,), activation="relu"))
kc.add(Dropout(.5))
kc.add(Dense(3, activation='softmax'))
kc.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

# kc = KerasClassifier(keras_model)
# kc = keras_model()
kc.fit(X_train.as_matrix(), pd.get_dummies(y_train).as_matrix(), validation_split=0.2, nb_epoch=50, batch_size=32, class_weight=cw)

predicted = kc.predict_classes(X_test.as_matrix())
actual = y_test.map({
        "functional": 0,
        "functional needs repair": 1,
        "non functional": 2
    }).values


print(pd.Series(predicted).value_counts())
accuracy_score(predicted, actual)
## Your input to keras needs to be input arrays, not dataframes.
# scores = cross_val_score(kc, cleaned_train[cols].as_matrix(), pd.get_dummies(train[TARGET]).as_matrix(), cv=5, fit_params = {'nb_epoch': 3})
# scores

# train_test_split(cleaned_train[cols], pd.get_dummies(train[TARGET]), test_size=0.33)


Train on 38016 samples, validate on 9504 samples
Epoch 1/50
38016/38016 [==============================] - 5s - loss: 1.7379 - acc: 0.4931 - val_loss: 1.4723 - val_acc: 0.5452
Epoch 2/50
38016/38016 [==============================] - 4s - loss: 1.5178 - acc: 0.4836 - val_loss: 1.4164 - val_acc: 0.5343
Epoch 3/50
38016/38016 [==============================] - 4s - loss: 1.4625 - acc: 0.4906 - val_loss: 1.4062 - val_acc: 0.5399
Epoch 4/50
38016/38016 [==============================] - 4s - loss: 1.4644 - acc: 0.4921 - val_loss: 1.3905 - val_acc: 0.5546
Epoch 5/50
38016/38016 [==============================] - 4s - loss: 1.4546 - acc: 0.4973 - val_loss: 1.3947 - val_acc: 0.5588
Epoch 6/50
38016/38016 [==============================] - 4s - loss: 1.4587 - acc: 0.4983 - val_loss: 1.3903 - val_acc: 0.5541
Epoch 7/50
38016/38016 [==============================] - 4s - loss: 1.4624 - acc: 0.4954 - val_loss: 1.4081 - val_acc: 0.5581
Epoch 8/50
38016/38016 [==============================] - 4s -

KeyboardInterrupt: 

{0: 1.0, 1: 7.482608695652174, 2: 1.4141331142152835}

In [181]:
predictions2 = kc.predict_classes(cleaned_train[cols].as_matrix())

59400/59400 [==============================] - 2s     


In [173]:
predictions2

array([2, 2, 2, ..., 2, 2, 2])

In [182]:
"""
1    34314
2    20514
0     4572
"""

"""
2    53609
0     5791
"""

pd.Series(predictions2).value_counts()

0    34221
2    18608
1     6571
dtype: int64

In [80]:
pd.Series(np.argmax(predictions, axis=1)).value_counts()

0    57881
2     1519
dtype: int64

In [87]:
pd.Series(np.argmax(predictions2, axis=1)).value_counts()

0    59400
dtype: int64

In [43]:
pd.DataFrame(predictions).sum()

0    59400.0
1        0.0
2        0.0
dtype: float32

In [48]:
import tflearn
# Classification
tflearn.init_graph(num_cores=8, gpu_memory_fraction=0.5)

net = tflearn.input_data(shape=[None, 290])
net = tflearn.fully_connected(net, 64)
net = tflearn.dropout(net, 0.5)
net = tflearn.fully_connected(net, 3, activation='softmax')
net = tflearn.regression(net, optimizer='adam', loss='categorical_crossentropy')

model = tflearn.DNN(net)
kc.fit(cleaned_train[cols].as_matrix(), pd.get_dummies(train[TARGET]).as_matrix(), validation_split=0.33, nb_epoch=3, batch_size=10)

ValueError: No variables to save

In [35]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.ensemble import BaggingClassifier
logistic = LogisticRegression()
print(cross_val_score(logistic, cleaned_train[cols], train[TARGET], n_jobs=-1).mean())
n_estimators=10
bagging = BaggingClassifier(logistic, max_samples=.2, n_estimators=n_estimators)
cross_val_score(bagging, cleaned_train[cols], train[TARGET], n_jobs=-1).mean()

0.745235690236


/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cann

0.74479797979797979

In [ ]:
from sklearn.linear_model import LogisticRegression

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier

n_estimators=100
svm = SVC(verbose=True, degree=5, kernel='poly')
bagging = BaggingClassifier(svm, max_samples=1.0 / n_estimators, n_estimators=n_estimators)
scores = cross_val_score(bagging, cleaned_train[cols], train[TARGET], n_jobs=-1)
scores

/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/Users/David/anaconda/envs/sci/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
Process ForkPoolWorker-15:
Process ForkPoolWorker-13:
Process ForkPoolWorker-14:
Process ForkPoolWorker-12:
Process ForkPoolWorker-16:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/David/anaconda/envs/sci/lib/python3.5/mult

KeyboardInterrupt: 

In [9]:
# add to kfkd.py
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet

net1 = NeuralNet(
    layers=[  # three layers: one hidden layer
        ('input', layers.InputLayer),
        ('hidden', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    # layer parameters:
    input_shape=(None, 290),  # 96x96 input pixels per batch
    hidden_num_units=100,  # number of units in hidden layer
    output_nonlinearity=None,  # output layer uses identity function
    output_num_units=30,  # 30 target values

    # optimization method:
    update=nesterov_momentum,
    update_learning_rate=0.01,
    update_momentum=0.9,

#     regression=True,  # flag to indicate we're dealing with regression problem
    max_epochs=400,  # we want to train this many epochs
    verbose=1,
    )

X, y = load()
net1.fit(X, y)

ImportError: cannot import name 'downsample'

In [20]:
len(cols)

290